In [5]:
import cdsapi
c = cdsapi.Client()
c.retrieve(
  'sis-heat-and-cold-spells',
  {
   'format': 'tgz',
   'variable': 'heat_wave_days',
   'definition': 'climatological_related',
   'experiment': ['rcp4_5', 'rcp8_5'],
   'ensemble_statistic': ['ensemble_members_average', 'ensemble_members_standard_deviation'],
  },
  'download.tar.gz')

2024-06-28 16:42:10,419 INFO Welcome to the CDS
2024-06-28 16:42:10,426 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-heat-and-cold-spells
2024-06-28 16:42:10,638 INFO Request is completed
2024-06-28 16:42:10,639 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data7/dataset-sis-heat-and-cold-spells-29bcb393-8f30-49f1-999c-abb4ee51f647.tar.gz to download.tar.gz (162.5M)
2024-06-28 16:42:22,947 INFO Download rate 13.2M/s                                                                     


Result(content_length=170370772,content_type=application/gzip,location=https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data7/dataset-sis-heat-and-cold-spells-29bcb393-8f30-49f1-999c-abb4ee51f647.tar.gz)

In [ ]:
pip install gdal

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
from osgeo import gdal
import numpy as np
from PIL import Image
import numexpr as ne
import matplotlib.pyplot as plt

# Origo -24,95 / 72,55 => Step 0.1°

# vvvvvvvvvvvvvvv
#
# Enter path to files here:
path = ''
#
# ^^^^^^^^^^^^^^^
#
#save bands as csv
def SaveAsCsv(nc_file, lat, lon):
    # Open the NetCDF file using GDAL
    dataset = gdal.Open(nc_file)

    if not dataset:
        raise FileNotFoundError(f"Cannot open NetCDF file: {nc_file}")

    # Get subdatasets (variables in the NetCDF file)
    subdatasets = dataset.GetSubDatasets()
    print(f"Found {len(subdatasets)} subdatasets (variables):")
    for sd in subdatasets:
        print(f" - {sd[0]}")

    # Assume we want to work with the first subdataset (variable)
    subdataset_name = subdatasets[0][0]
    subdataset = gdal.Open(subdataset_name)

    cols = subdataset.RasterXSize
    rows = subdataset.RasterYSize

    print (f"{rows}/{cols}")
    for i in range(100):
        band = dataset.GetRasterBand(i+1)
        data = band.ReadAsArray(0, 0, cols, rows)
        np.savetxt(f'{path}/array.csv', data, delimiter=',', fmt='%s')

# Function to read and filter NetCDF data
def filter_nc_by_location(nc_file, lat, lon):
    # Open the NetCDF file using GDAL
    dataset = gdal.Open(nc_file)

    if not dataset:
        raise FileNotFoundError(f"Cannot open NetCDF file: {nc_file}")

    # Get subdatasets (variables in the NetCDF file)
    subdatasets = dataset.GetSubDatasets()
    print(f"Found {len(subdatasets)} subdatasets (variables):")
    for sd in subdatasets:
        print(f" - {sd[0]}")

    # Assume we want to work with the first subdataset (variable)
    subdataset_name = subdatasets[0][0]
    subdataset = gdal.Open(subdataset_name)

    cols = subdataset.RasterXSize
    rows = subdataset.RasterYSize

    y = int(ne.evaluate(f"({lon}+24.95)/.1"))
    x = int(ne.evaluate(f"-({lat}-72.55)/.1"))
    a = []
    print (f"{x}/{y}")
    for i in range(100):
        band = dataset.GetRasterBand(i+1)
        data = band.ReadAsArray(0, 0, cols, rows)
        a.append(data[x][y])

    # Read latitude and longitude arrays
    #lat_var = subdataset.GetRasterBand(1)
    #lon_var = subdataset.GetRasterBand(2)

    #lat_data = lat_var.ReadAsArray()
    #lon_data = lon_var.ReadAsArray()

    # Filter based on the provided lat/lon range
    #mask = (lat_data >= lat_range[0]) & (lat_data <= lat_range[1]) & \
    #       (lon_data >= lon_range[0]) & (lon_data <= lon_range[1])

    # Apply mask to data (example for 2D dataset)
    #filtered_data = lat_var.ReadAsArray()[mask]

    return a

# Example usage
def main():
    lon = 47.66
    lat = 9.17
    nc_file = f"{path}/HWD_EU_climate_rcp45_mean_v1.0.nc"
    data45 = filter_nc_by_location(nc_file, lon, lat)

    nc_file = f"{path}/HWD_EU_climate_rcp85_mean_v1.0.nc"
    data85 = filter_nc_by_location(nc_file, lon, lat)
    
    nc_file = f"{path}/HWD_EU_climate_rcp45_stdev_v1.0.nc"
    st45 = filter_nc_by_location(nc_file, lon, lat)
    
    nc_file = f"{path}/HWD_EU_climate_rcp85_stdev_v1.0.nc"
    st85 = filter_nc_by_location(nc_file, lon, lat)

    #print(f"Filtered data shape: {filtered_data.shape}")
    
    x_values = np.arange(1986,1986+len(data45))
    #plt.figure(figsize=(10, 6))  # Set the figure size
    #plt.plot(x_values, data45, marker='o', linestyle='-', color='b', label='Data Line')
    #plt.plot(x_values, data85, marker='o', linestyle='-', color='r', label='Data Line')
    #plt.errorbar(x_values, data45,st45, marker='', linestyle='-', color='b', label='Data Line', capsize=1.5)
    #plt.errorbar(x_values, data85,st85, marker='', linestyle='-', color='r', label='Data Line', capsize=1.5)
    
#####################
    modelname = "RCP 4.5 / 8.5"
    date_vals = np.arange(1986,1986+len(data45))

    plt.plot(date_vals,data45,color='b')
    plt.errorbar(date_vals[::2],data45[::2],st45[::2],color='b',capsize=3) #-(rcp_choice-4.5)/45
    plt.errorbar([],[],[],color='b',label='RCP 4.5',capsize=1.5)
    
    plt.plot(date_vals,data85,color='r')
    plt.errorbar(date_vals[::2],data85[::2],st85[::2],color='r',capsize=3) #-(rcp_choice-4.5)/45
    plt.errorbar([],[],[],color='r',label='RCP 8.5',capsize=1.5)


    plt.xlabel('Jahr',fontsize=12)
    plt.ylabel('Anzahl Hitzetage pro Jahr',fontsize=12)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.title('Hitzewellen-Prognose ' + modelname,fontweight='bold')
    
    
    plt.grid(axis='y')
    plt.legend(fontsize=12,loc='upper left')
    plt.xlim(min(date_vals),max(date_vals))
    plt.ylim(0,None)
    plt.savefig(f"{path}/rcp_compare.png")
    print("Saving figure to rcp_compare.png")
    plt.show()
main()

FileNotFoundError: Cannot open NetCDF file: /HWD_EU_climate_rcp45_mean_v1.0.nc